# Compare mock communities from the NextSeq2000 run 

These mock communities have been used as positive controls within the RxCS project (see analysis here: https://github.com/LenaFloerl/RxCS.git).

These were already demultiplexed, denoised with dada2 and taxonomically classified with UNITE database (v9.0) and SILVA (99%, release 138, trained for the 16S  515F-806R (V4) region)

In [6]:
import qiime2 as q2
import qiime2.plugins.taxa.actions as taxa_actions
from qiime2 import Visualization, Metadata
from qiime2.plugins.taxa.methods import collapse
from qiime2.plugins.feature_table.methods import relative_frequency 
from biom import load_table
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.patches as mpatches


%matplotlib inline

# ITS 

In [127]:
wd = '/home/lfloerl/cloud/lfloerl/RxCS_FGCZ/artifacts/ITS-fwd'
%cd $wd 

/home/lfloerl/cloud/lfloerl/RxCS_FGCZ/artifacts/ITS-fwd


In [128]:
metadata = '/home/lfloerl/RxCS/Metadata/ITS_metadata.tsv'
md = Metadata.load(metadata)

taxonomy = q2.Artifact.load('taxonomy.qza')

### Get positive controls 

In [129]:
%%bash 

qiime feature-table filter-samples \
  --i-table decontam-table.qza \
  --m-metadata-file /home/lfloerl/RxCS/Metadata/ITS_metadata.tsv \
  --p-where "[control]='pos_ctrl' AND [Sample_ID] IN ('DNA18-pos-1','DNA19-pos-1','DNA7-pos-1','DNA18-pos-2','DNA23-pos-1','DNA5-pos-1','DNA20-pos-1')" \
  --o-filtered-table pos-ctrl-table.qza

Saved FeatureTable[Frequency] to: pos-ctrl-table.qza


In [130]:
table = q2.Artifact.load('pos-ctrl-table.qza')

### Make a taxa barplot

In [131]:
taxa_bar_plots_viz, = taxa_actions.barplot(
    table=table,
    taxonomy=taxonomy,
    metadata=md
)
taxa_bar_plots_viz.save('pos-ctrl-taxa_barplot.qzv')

'pos-ctrl-taxa_barplot.qzv'

In [132]:
Visualization.load('pos-ctrl-taxa_barplot.qzv')

<visualization: Visualization uuid: 3fafbeab-a14d-4979-8e03-98793b310636>

In [77]:
# collapse table at taxonomic level 7 (ASVs)
collapsed_table, = collapse(table=table,
                           taxonomy=taxonomy,
                           level=7)

# get the relative frequencies 
relative_freq_collapsed, = relative_frequency(table=collapsed_table)
relative_freq_collapsed.save('relative_freq_collapsed.qza')

# export 
!qiime tools export --input-path relative_freq_collapsed.qza --output-path relative_freq_collapsed

Exported relative_freq_collapsed.qza as BIOMV210DirFmt to directory relative_freq_collapsed


In [78]:
# convert the biom table to a df
def biom_to_dataframe(biom_file):
    table = load_table(biom_file)
    df = pd.DataFrame(table.matrix_data.toarray(), 
                      index=table.ids(axis='observation'), 
                      columns=table.ids(axis='sample'))
    return df

df_ITS = biom_to_dataframe('relative_freq_collapsed/feature-table.biom')

# metadata
md_df = md.to_dataframe()

In [79]:
# rename the cols to the sample names 
column_mapping = md_df['Sample_ID'].to_dict()
df_ITS = pd_biom.rename(columns=column_mapping)

In [80]:
df_ITS.head()

,DNA18-pos-1_noPNAs_0,DNA18-pos-1_mPNA_0.25,DNA18-pos-1_mPNA_0.5,DNA18-pos-1_mPNA_0.75,DNA18-pos-1_mPNA_1,DNA18-pos-1_mPNA_1.25,DNA18-pos-1_pPNA_0.25,DNA18-pos-1_pPNA_0.5,DNA18-pos-1_pPNA_0.75,DNA18-pos-1_pPNA_1,...,DNA18-pos-4,DNA21-pos-4,DNA21-pos-3,DNA22-pos-4,DNA22-pos-3,DNA22-pos-2,DNA23-pos-1,DNA18-pos-1_m+pPNA_0.75,DNA18-pos-1_m+pPNA_1,DNA18-pos-1_m+pPNA_1.25
d__Bacteria;p__Firmicutes;c__Bacilli;o__Bacillales;f__Bacillaceae;g__Bacillus;__,0.235959,0.232901,0.235239,0.235016,0.232221,0.229852,0.231567,0.225628,0.236547,0.238857,...,0.372263,0.08125,0.289517,0.142560,0.16939,0.237922,0.319685,0.234261,0.234518,0.231764
d__Bacteria;p__Actinobacteriota;c__Actinobacteria;o__Frankiales;f__Nakamurellaceae;g__Nakamurella;s__uncultured_bacterium,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.008319,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
d__Bacteria;p__Firmicutes;c__Bacilli;o__Bacillales;f__Bacillaceae;__;__,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.046589,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
d__Bacteria;p__Firmicutes;c__Bacilli;o__Paenibacillales;f__Paenibacillaceae;g__uncultured;__,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.029136,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
d__Bacteria;p__Proteobacteria;c__Alphaproteobacteria;o__Acetobacterales;f__Acetobacteraceae;g__Roseomonas;s__metagenome,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000


# 16S 

In [11]:
wd = '/home/lfloerl/cloud/lfloerl/RxCS_FGCZ/artifacts/16S'
%cd $wd 

/home/lfloerl/cloud/lfloerl/RxCS_FGCZ/artifacts/16S


In [17]:
!ls bac-dada2/dada-rep-seqs.qza

dada-rep-seqs-evaluate.qzv  dada-rep-seqs.qzv  dada-stats.qza  dada-table.qza
dada-rep-seqs.qza	    dada-stats	       dada-stats.qzv  dada-table.qzv


In [12]:
metadata = '/home/lfloerl/RxCS/Metadata/16S_metadata.tsv'
md = Metadata.load(metadata)

taxonomy = q2.Artifact.load('taxonomy.qza')

In [13]:
%%bash 

qiime feature-table filter-samples \
  --i-table chloroplast_mitochondria_filtered_table.qza \
  --m-metadata-file /home/lfloerl/RxCS/Metadata/16S_metadata.tsv \
  --p-where "[control]='pos_ctrl' AND [Sample_ID] IN ('DNA18-pos-1','DNA19-pos-1','DNA7-pos-1','DNA18-pos-2','DNA23-pos-1','DNA5-pos-1','DNA20-pos-1')" \
  --o-filtered-table pos-ctrl-table-ContamFiltered.qza

Saved FeatureTable[Frequency] to: pos-ctrl-table-ContamFiltered.qza


In [14]:
table = q2.Artifact.load('pos-ctrl-table-ContamFiltered.qza')

Taxa Barplot

In [15]:
taxa_bar_plots_viz, = taxa_actions.barplot(
    table=table,
    taxonomy=taxonomy,
    metadata=md
)
taxa_bar_plots_viz.save('pos-ctrl-taxa_barplot-ContamFiltered.qzv')

'pos-ctrl-taxa_barplot-ContamFiltered.qzv'

In [126]:
Visualization.load('pos-ctrl-taxa_barplot-ContamFiltered.qzv')

<visualization: Visualization uuid: da4cc794-ed42-4401-b8ae-6978865a8110>

Export

In [87]:
# collapse table at taxonomic level 7 (ASVs)
collapsed_table, = collapse(table=table,
                           taxonomy=taxonomy,
                           level=7)

# get the relative frequencies 
relative_freq_collapsed, = relative_frequency(table=collapsed_table)
relative_freq_collapsed.save('relative_freq_collapsed.qza')

# export 
!qiime tools export --input-path relative_freq_collapsed.qza --output-path relative_freq_collapsed

Exported relative_freq_collapsed.qza as BIOMV210DirFmt to directory relative_freq_collapsed


In [88]:
# convert the biom table to a df
def biom_to_dataframe(biom_file):
    table = load_table(biom_file)
    df = pd.DataFrame(table.matrix_data.toarray(), 
                      index=table.ids(axis='observation'), 
                      columns=table.ids(axis='sample'))
    return df

df_16S = biom_to_dataframe('relative_freq_collapsed/feature-table.biom')

# metadata
md_df = md.to_dataframe()

In [89]:
# rename the cols to the sample names 
column_mapping = md_df['Sample_ID'].to_dict()
df_16S = pd_biom.rename(columns=column_mapping)

In [90]:
df_16S.head()

,DNA18-pos-1_noPNAs_0,DNA18-pos-1_mPNA_0.25,DNA18-pos-1_mPNA_0.5,DNA18-pos-1_mPNA_0.75,DNA18-pos-1_mPNA_1,DNA18-pos-1_mPNA_1.25,DNA18-pos-1_pPNA_0.25,DNA18-pos-1_pPNA_0.5,DNA18-pos-1_pPNA_0.75,DNA18-pos-1_pPNA_1,...,DNA18-pos-4,DNA21-pos-4,DNA21-pos-3,DNA22-pos-4,DNA22-pos-3,DNA22-pos-2,DNA23-pos-1,DNA18-pos-1_m+pPNA_0.75,DNA18-pos-1_m+pPNA_1,DNA18-pos-1_m+pPNA_1.25
d__Bacteria;p__Firmicutes;c__Bacilli;o__Bacillales;f__Bacillaceae;g__Bacillus;__,0.235959,0.232901,0.235239,0.235016,0.232221,0.229852,0.231567,0.225628,0.236547,0.238857,...,0.372263,0.08125,0.289517,0.142560,0.16939,0.237922,0.319685,0.234261,0.234518,0.231764
d__Bacteria;p__Actinobacteriota;c__Actinobacteria;o__Frankiales;f__Nakamurellaceae;g__Nakamurella;s__uncultured_bacterium,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.008319,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
d__Bacteria;p__Firmicutes;c__Bacilli;o__Bacillales;f__Bacillaceae;__;__,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.046589,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
d__Bacteria;p__Firmicutes;c__Bacilli;o__Paenibacillales;f__Paenibacillaceae;g__uncultured;__,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.029136,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
d__Bacteria;p__Proteobacteria;c__Alphaproteobacteria;o__Acetobacterales;f__Acetobacteraceae;g__Roseomonas;s__metagenome,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000


# Merge!

In [96]:
df_ITS.shape

(225, 37)

In [97]:
df_16S.shape

(225, 37)

In [98]:
df_combined = pd.concat([df_ITS, df_16S], axis=0)
df_combined

,DNA18-pos-1_noPNAs_0,DNA18-pos-1_mPNA_0.25,DNA18-pos-1_mPNA_0.5,DNA18-pos-1_mPNA_0.75,DNA18-pos-1_mPNA_1,DNA18-pos-1_mPNA_1.25,DNA18-pos-1_pPNA_0.25,DNA18-pos-1_pPNA_0.5,DNA18-pos-1_pPNA_0.75,DNA18-pos-1_pPNA_1,...,DNA18-pos-4,DNA21-pos-4,DNA21-pos-3,DNA22-pos-4,DNA22-pos-3,DNA22-pos-2,DNA23-pos-1,DNA18-pos-1_m+pPNA_0.75,DNA18-pos-1_m+pPNA_1,DNA18-pos-1_m+pPNA_1.25
d__Bacteria;p__Firmicutes;c__Bacilli;o__Bacillales;f__Bacillaceae;g__Bacillus;__,0.235959,0.232901,0.235239,0.235016,0.232221,0.229852,0.231567,0.225628,0.236547,0.238857,...,0.372263,0.08125,0.289517,0.142560,0.16939,0.237922,0.319685,0.234261,0.234518,0.231764
d__Bacteria;p__Actinobacteriota;c__Actinobacteria;o__Frankiales;f__Nakamurellaceae;g__Nakamurella;s__uncultured_bacterium,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.008319,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
d__Bacteria;p__Firmicutes;c__Bacilli;o__Bacillales;f__Bacillaceae;__;__,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.046589,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
d__Bacteria;p__Firmicutes;c__Bacilli;o__Paenibacillales;f__Paenibacillaceae;g__uncultured;__,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.029136,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
d__Bacteria;p__Proteobacteria;c__Alphaproteobacteria;o__Acetobacterales;f__Acetobacteraceae;g__Roseomonas;s__metagenome,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
d__Bacteria;p__Actinobacteriota;c__Actinobacteria;o__Streptosporangiales;f__Thermomonosporaceae;g__Actinoallomurus;s__Actinoallomurus_sp.,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
d__Bacteria;p__Firmicutes;c__Bacilli;o__Alicyclobacillales;f__Alicyclobacillaceae;g__Tumebacillus;s__Tumebacillus_permanentifrigoris,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.056572,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
d__Bacteria;p__Firmicutes;c__Bacilli;o__Alicyclobacillales;f__Alicyclobacillaceae;g__Alicyclobacillus;s__Alicyclobacillus_herbarius,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
d__Bacteria;p__Chloroflexi;c__KD4-96;o__KD4-96;f__KD4-96;g__KD4-96;__,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
